In [1]:
import pandas as pd

import numpy as np

import xlwings as xw

In [2]:
def crr_put_option(S, K, T, r, sigma, q, n=1000):
    dt = T / n
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp((r - q) * dt) - d) / (u - d)
    
    # Initialize stock price tree

    stock_tree = np.zeros((n + 1, n + 1))
    stock_tree[0, 0] = S
    for i in range(1, n + 1):
        stock_tree[i, 0] = stock_tree[i - 1, 0] * u
        for j in range(1, i + 1):
            stock_tree[i, j] = stock_tree[i - 1, j - 1] * d
            
    # Initialize option value tree
    option_tree = np.zeros((n + 1, n + 1))
    for j in range(n + 1):
        option_tree[n, j] = max(K - stock_tree[n, j], 0)
        
    # Backward induction to calculate option value
    for i in range(n - 1, -1, -1):
        for j in range(i + 1):
            option_tree[i, j] = max(K - stock_tree[i, j], np.exp(-r * dt) * (p * option_tree[i + 1, j] + (1 - p) * option_tree[i + 1, j + 1]))

    return option_tree[0, 0]
 

In [4]:
opt_value = crr_put_option(10, 10, 5, 4, 3, 1, n=100)

In [5]:
print(opt_value)

2.9120315916375397


In [22]:
data = pd.read_excel("Desktop/opt_value_test_60_1.xlsx")

In [28]:
print(data.head())

   Harvest Age            QT  pm_60_percent_put          Pr  Decision   T  \
0           40  47453.236009         232.491146  236.045987         1  30   
1           33  35845.370305         253.633933  268.698706         1  37   
2           49  60500.575917         217.607145  219.550813         1  21   
3           30  30536.154886         269.070905  297.237228         1  40   
4           25  21461.158524         312.505580  359.307213         1  45   

      r  volatility  div_rate       S  Option Value  
0  0.04      0.3885         0  169.19     50.028807  
1  0.04      0.3885         0  169.19     47.442847  
2  0.04      0.3885         0  169.19     49.608639  
3  0.04      0.3885         0  169.19     45.102359  
4  0.04      0.3885         0  169.19     40.944255  


In [29]:
# Calculate the American put option value with the CRR method and store it in the 'Option Value' column

data["Option Value"] = data.apply(lambda row: crr_put_option(row["Pr"],
                                                            row["S"],
                                                            row["T"],
                                                            row["r"],
                                                            row["volatility"],
                                                            row["div_rate"]), axis=1)

In [30]:
print(data.head())

   Harvest Age            QT  pm_60_percent_put          Pr  Decision   T  \
0           40  47453.236009         232.491146  236.045987         1  30   
1           33  35845.370305         253.633933  268.698706         1  37   
2           49  60500.575917         217.607145  219.550813         1  21   
3           30  30536.154886         269.070905  297.237228         1  40   
4           25  21461.158524         312.505580  359.307213         1  45   

      r  volatility  div_rate       S  Option Value  
0  0.04      0.3885         0  169.19     50.028807  
1  0.04      0.3885         0  169.19     47.442847  
2  0.04      0.3885         0  169.19     49.608639  
3  0.04      0.3885         0  169.19     45.102359  
4  0.04      0.3885         0  169.19     40.944255  
